In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
# Always make it pretty.
plt.style.use('ggplot')
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import datetime

In [2]:
file = pd.read_csv('data/Collisions.csv')

In [3]:
file[file['SPEEDING'] == 'Y'].shape[0]
#Number of Accidents with speeding involved

9492

In [4]:
file[(file['SPEEDING'] == 'Y') & (file['FATALITIES'] > 0)].shape[0]

85

In [5]:
file[file['FATALITIES'] > 0].shape[0]

328

In [6]:
print(file['INCDTTM'].isnull().sum())
file['UNDERINFL'].isnull().sum()


0


25721

In [7]:
file[(file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1' )].shape[0]

9315

In [8]:
file[file['SERIOUSINJURIES'] > 0].shape[0]
file['UNDERINFL'].unique()
drunkdracc = file[((file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1'))].shape[0]

In [9]:
infl = file[((file['UNDERINFL'] == 'Y') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['FATALITIES'] > 0))].shape[0]


underinflratio = infl / file[file['FATALITIES'] > 0 ].shape[0]

noninfl = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]

noninfluratio = noninfl / file[file['FATALITIES'] > 0].shape[0]
#print(noninfluratio)
#print(underinflratio)
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Block')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Intersection')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['LIGHTCOND'] == 'Daylight')].shape[0])
print('drunk driving deaths / drunk driving accidents: ',(infl/drunkdracc))
nondrunkdrivingacccsdeath = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]
nondrunkdrivingacc = file[(file['UNDERINFL'] == 'N') | (file['UNDERINFL'] == '0')].shape[0]
print(nondrunkdrivingacc)
print('non drunk driving deaths / non drunk driving accidents: ',(nondrunkdrivingacccsdeath/nondrunkdrivingacc))
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc
shared_sample_freq = (nondrunkdrivingacccsdeath + infl) / denominator
shared_sample_variance = numerator * (shared_sample_freq * (1 - shared_sample_freq)) / denominator
difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))
difference_in_sample_proportions = (nondrunkdrivingacccsdeath/nondrunkdrivingacc)/(infl/drunkdracc)

p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print(p_value)

drunk driving deaths / drunk driving accidents:  0.00966183574879227
177568
non drunk driving deaths / non drunk driving accidents:  0.001267120201838169
0.0


In [10]:
seriousinjury = file[((file['UNDERINFL'] == 'Y') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['SERIOUSINJURIES'] > 0))].shape[0]
totalseriousinjury = file[file['SERIOUSINJURIES'] > 0].shape[0]
seriousinjurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['SERIOUSINJURIES'] > 0))].shape[0]
print(seriousinjury/totalseriousinjury)
print('serious injury driving accidents invoving drunk / drunk driving accidents: ',seriousinjury/drunkdracc)
print('serious injury driving accidents not involving drunk/ non drunk driving accidents: ',seriousinjurynondrunk/nondrunkdrivingacc)

0.14145516074450085
serious injury driving accidents invoving drunk / drunk driving accidents:  0.04487385936661299
serious injury driving accidents not involving drunk/ non drunk driving accidents:  0.014276220940710039


In [11]:
injury = file[((file['UNDERINFL'] == 'Y') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['INJURIES'] > 0))].shape[0]
injurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['INJURIES'] > 0))].shape[0]
totalinjury = file[file['INJURIES'] > 0].shape[0]
print(injury/totalinjury)
print(injury/drunkdracc)
print(injurynondrunk/nondrunkdrivingacc)

0.06567435901758172
0.41825013419216317
0.3051563344746801


In [12]:
print('car accidents involving drunk driver that injury someone',injury)
print('car accidents involving non drunk driver that injury someone',injurynondrunk)
print('car accidents involving drunk driver',drunkdracc)
print('car accidents involving sober driver',nondrunkdrivingacc)
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc

car accidents involving drunk driver that injury someone 3896
car accidents involving non drunk driver that injury someone 54186
car accidents involving drunk driver 9315
car accidents involving sober driver 177568


In [13]:

#file['ST_COLCODE'] = pd.to_numeric(file[file.ST_COLCODE.notnull()])
file['ST_COLCODE'] = pd.to_numeric(file['ST_COLCODE'],errors = 'coerce')
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())

115
191
823
2050
6673
50121


In [15]:
latlong= file.iloc[0:101,0:2]
latlong['state'] = 'WA'
latlong
#trying to get all the zipcodes of data

,X,Y,state
0,-122.270834,47.539355,WA
1,-122.306332,47.606167,WA
2,-122.269611,47.516849,WA
3,-122.307505,47.612898,WA
4,-122.312430,47.708528,WA
5,-122.328079,47.604161,WA
6,-122.368598,47.670894,WA
7,-122.344507,47.687665,WA
8,-122.334266,47.733225,WA
9,-122.380504,47.672365,WA


In [87]:
file2 = file.copy()
#file2['INCDTTM'].str.contains('AM|PM',regex=True)
#test2 =file2['INCDTTM']
#test3 =test2[file2['INCDTTM'].str.contains('AM|PM',regex=True)]
#print(type(datetime.datetime.strptime('7/4/2019','%m/%d/%Y')))

In [42]:
for idx, my_date in enumerate(test2):
    try:
        file2.loc[idx,'INCDTTM'] = datetime.datetime.strptime(my_date,'%m/%d/%Y %I:%M:%S %p')
    except:
        file2.loc[idx,'INCDTTM'] = datetime.datetime.strptime(my_date,'%m/%d/%Y')

In [38]:
for i in range(212604):
    if not isinstance(file2.loc[i,'INCDTTM'], datetime.datetime):
        print(i)
        break

7674


In [ ]:
file

In [81]:
#file2.to_csv('data/CollionsWDatetime.csv',index=False)
file3 = pd.read_csv('data/CollionsWDatetime.csv')
#212604

In [82]:
#file3 = file3.dropna(subset=['INCDTTM'])
#file3.shape[0]
sum(file3['INCDTTM'].isnull())

0

In [64]:
file2.to_pickle('data/CollionsWDatetime.pkl.bz2', compression = 'bz2')

In [83]:
def datetimeconversion(row):
    try:
        return datetime.datetime.strptime(row['INCDTTM'],'%m/%d/%Y %I:%M:%S %p')
    except:
        return datetime.datetime.strptime(row['INCDTTM'],'%m/%d/%Y')

In [91]:
file2['INCDTTM']=file2.apply(datetimeconversion, axis=1)


<class 'pandas._libs.tslib.Timestamp'>


In [159]:
file2.loc[0:5,'INCDTTM']

0   2019-07-04 00:00:00
1   2019-07-10 00:00:00
2   2019-04-19 18:21:00
3   2019-07-10 00:00:00
4   2019-04-19 16:16:00
5   2019-05-08 07:32:00
Name: INCDTTM, dtype: datetime64[ns]

In [150]:
test3.min()

datetime.datetime(2004, 1, 10, 1, 27)

200

In [200]:
dic = json.loads(r.content)
type(dic)

dict

In [226]:
def getweather():
url = 'https://api.darksky.net/forecast/'
with open('data/dark_sky.txt', 'r') as file:
    key = file.read()
latlong = '/47.44472,-122.31361,'
for i in range(5683): #5682 is number of days from beg to end
    time = 1389346020 + (i*24*60*60)
    strtime = str(time)
    finalurl= url+key+latlong+strtime
    r = requests.get(finalurl)
    dic = json.loads(r.content)
    Weather.insert_one(dic)
 
    

In [251]:
friends_of_twilight = testcoll.find({})
i = 0
for friend in friends_of_twilight:
    if i > 0:
        break
    pprint.pprint(friend['currently']['humidity'])
    i += 1

0.74


In [237]:
datetime.datetime.utcfromtimestamp(1389340800)-datetime.timedelta(hours=8)

datetime.datetime(2014, 1, 10, 8, 0)

In [93]:
def getprecipitation(Weather):
    times = Weather.find({})
    info = dict()
    for time in times:
        for i in range(len(time['hourly']['data'])):
            t = time['hourly']['data'][i]['time']
            try: 
                rain = time['hourly']['data'][i]['precipIntensity']
            except:
                continue
            dt = datetime.datetime.utcfromtimestamp(t)-datetime.timedelta(hours=8)
            info[dt] = rain
    return info

In [99]:
rain = getprecipitation(Weather)
rainpd = pd.DataFrame.from_dict(rain, orient='index')


In [227]:
def thirtysixhoursofnorain(df,num):
    count = 0
    lst = []
    for i, index in zip(range(df.size),df.index):
        if df.iloc[i,0] == 0:
            count += 1
        if df.iloc[i,0] != 0:
            if count >= num:
                lst.append(index)
            count = 0
    return lst

In [206]:
thirtysixhourlst = thirtysixhoursofnorain(rainpd)
twentyfourhourdrylst = twentyfourhourdry(rainpd)

In [228]:
seventytwohourlist = thirtysixhoursofnorain(rainpd,72)

147

In [231]:
def getaccidentsdict(df,lst):
    diction = dict()    
    for i in lst:
        accidents = df[(df['INCDTTM'].dt.time != datetime.time(0,0)) & (df['INCDTTM']>= i.to_pydatetime()) & 
                          (df['INCDTTM']<= i.to_pydatetime()+ datetime.timedelta(hours=1))].loc[:,'INCDTTM'].count()
        if accidents > 0:
            diction[i.to_pydatetime()] = accidents
    return diction
def getdrydrivingdict(df,lst):
    diction = dict()    
    for i in lst:
        accidents = df[(df['INCDTTM'].dt.time != datetime.time(0,0)) & (df['INCDTTM']<= i.to_pydatetime() - 
                                                                        datetime.timedelta(hours=2)) & 
                       (df['INCDTTM']>= i.to_pydatetime()- datetime.timedelta(hours=3))].loc[:,'INCDTTM'].count()
        if accidents > 0:
            diction[i.to_pydatetime()] = accidents
    return diction

In [208]:
afterthirtysixhourdry = getaccidentsdict(file2,thirtysixhourlst)

In [222]:
afterthirtysixhourrealdry = getdrydrivingdict(file2,thirtysixhourlst)

461
191


In [232]:
afterseventytwohourwet = getaccidentsdict(file2,seventytwohourlist)
aftersevnetytwohourdry = getdrydrivingdict(file2,seventytwohourlist)

In [233]:
print(sum(afterseventytwohourwet.values()))
print(len(afterseventytwohourwet.values()))

248
100


In [234]:
print(sum(aftersevnetytwohourdry.values()))
print(len(aftersevnetytwohourdry.values()))

197
96


In [223]:
print(sum(afterthirtysixhourdry.values()))
print(len(afterthirtysixhourdry.values()))

461
191


In [224]:
print(sum(afterthirtysixhourrealdry.values()))
print(len(afterthirtysixhourrealdry.values()))

348
172


In [198]:
i=(thirtysixhourlst[1])
file2[(file2['INCDTTM'].dt.time != datetime.time(0,0)) & (file2['INCDTTM']>= i.to_pydatetime()) & 
                      (file2['INCDTTM']<= i.to_pydatetime()+ datetime.timedelta(hours=1))].loc[:,'INCDTTM'].count()

2

In [176]:
pd.to_datetime('00:00:00').dt.time

AttributeError: 'Timestamp' object has no attribute 'dt'

In [194]:
(file2['INCDTTM'].dt.time.loc[0])

datetime.time(0, 0)

In [195]:
datetime.time(0,0)

datetime.time(0, 0)

In [ ]:
def findwetafterdry(lst,df):

In [95]:
from pymongo import MongoClient
import pprint

In [96]:
client = MongoClient('localhost', 27017)

In [97]:
db = client['WeatherDB']

In [98]:
Weather = db['Weather']

In [194]:
testcoll = db['testcoll']
PrecpIntAndProbHourly = db['PrecpIntAndProbHourly']

(1, 2)